# Freeciv Connection

For this iteration I am trying to connect to the freeciv-desktop server running locally on my machine. This notebook is written to establish rules for connection and how to work further in establishing the communication side. Once this part is established we can proceed to using of information and then eventual deep learning support. This also happens to be the first step and thus the name `1_Connection.ipynb`

## Goals

Goals for this notebook are simple:
1. establish connection to freeciv-server running locally
2. transfer relevant information and document the signals
3. establish how to use those signals in the most effective manner possible

## LOGS

**29.12.2018:** Start writing this notebook.

## Basics

What I need to first start with is learning how to communicate with the locally running server. In another implementation for connecting to freeciv server that I read `tornado` was used to communicate, though that might have handled the job well, this thing is very new to me. I am starting by using sockets, which is what is used to connect two nodes. One node listens to a particular IP and node while the other node sends out information to connect to the node.

Here we first make the socket, then resolve host IP and then connect to it from a particular port. [IBM](https://developer.ibm.com/tutorials/l-pysocks/) has a great tutorial on it.

In [1]:
# importing the dependencies
import socket

In [3]:
# establish socket
try: 
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
    print("Socket successfully created")
except socket.error as err: 
    print("socket creation failed with error:", err) 
    
# default port for socket 
port = 80
  
try: 
    host_ip = socket.gethostbyname('www.ibm.com')
except socket.gaierror:
    # this means could not resolve the host 
    print("there was an error resolving the host")
    
# connecting to the server 
s.connect((host_ip, port)) 

print("the socket has successfully connected to IBM on port ==", host_ip)
s.close()

Socket successfully created
the socket has successfully connected to IBM on port == 104.120.84.241


### Connecting to freeciv

I have a freeciv-desktop server running locally on my system at `port 5000` this is what I am genereally running it on to avoid any ambiguity. Note that the `--debug=4` and we are getting information from the log dumps, one for server and one for client.

In [7]:
# establish socket
try: 
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
except socket.error as err: 
    print("socket creation failed with error:", err)
    
# port
PORT_FC = 5000

# connection
s.connect((socket.gethostbyname('localhost'), PORT_FC))

We were able to connect and are getting response like this (from server log):

```
4: in server_sniff_all_input() [sernet.c::766]: got new connection
4: in server_make_connection() [sernet.c::1073]: connection (c8) from localhost (127.0.0.1)
4: in server_sniff_all_input() [sernet.c::629]: connection (c8 from localhost (connection incomplete)) cut due to ping timeout
```

#### PACKET STRUCTURE

All the packet related information is given in the file `packets.def` including structure of different packets and in the order the packets should be sent. This notebook has many details directly copied from that file. The first packet that is sent for connection is `PACKET_SERVER_JOIN_REQ` and the structure is as follows:

```
PACKET_SERVER_JOIN_REQ = 4; cs, dsend, no-delta, no-handle
    STRING username[48];
    STRING capability[512];
    STRING version_label[48];
    UINT32 major_version, minor_version, patch_version;
end
```

So let's try to make a similar packet in python and then send it over to server. Problem right now is finding the `capability` and other such parameters, one way to solve this is by brute force searching for it using search option in our GUI systems.By doing this I found that
```
The actual capability string is now defined in fc_version = "3.0.91-dev"
version_label = "-dev"
major_version = 3
minor_version = 0
patch_version = 91
```
Version information is given in `fc_version` file.

In [8]:
username = 'pehla'
capability = '3.0.91-dev'
version_label = '-dev'
major_version = '3'
minor_version = '0'
patch_version = '91'

fields = [username, capability, version_label, major_version, minor_version, patch_version]

text_ = bytes(' '.join(fields), 'utf-8')

print(text_)

b'pehla 3.0.91-dev -dev 3 0 91'


In [9]:
s.sendall(text_)

In [5]:
packet = b'''PACKET_SERVER_JOIN_REQ = 4; cs, dsend, no-delta, no-handle
STRING "pehla";
STRING "3.0.91-dev";
STRING "-dev";
UINT32 3, 0, 91;
end
'''

In [13]:
print(packet)

b'PACKET_SERVER_JOIN_REQ = 4; cs, dsend, no-delta, no-handle\nSTRING "pehla";\nSTRING "3.0.91-dev";\nSTRING "-dev";\nUINT32 3, 0, 91;\nend\n'


In [10]:
s.sendall(packet)

In [15]:
d = s.recv(10000)

In [16]:
print(d)

b''
